In [ ]:
import os
import pathlib
import shapely.geometry
import geopandas
import shutil

# Define data paths

In [ ]:
data_dir = pathlib.Path(os.getcwd()) / pathlib.Path('data')
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
catchment_name = 'catchment_boundary'
catchment_dir = data_dir / pathlib.Path(catchment_name)
land_name = 'land'
land_dir = data_dir / pathlib.Path(land_name)

# Create catchment and land boundary

In [ ]:
x0 = 250; y0 = -250; x1 = 1250; y1 = 750;
catchment = shapely.geometry.Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])
catchment = geopandas.GeoSeries([catchment])
catchment.to_file(catchment_dir)
shutil.make_archive(base_name=catchment_dir, format='zip', root_dir=data_dir, base_dir=catchment_name)
shutil.rmtree(catchment_dir)

In [ ]:
x0 = 0; y0 = -0; x1 = 1500; y1 = 1000;
land = shapely.geometry.Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])
land = geopandas.GeoSeries([land])
catchment.to_file(land_dir)
shutil.make_archive(base_name=land_dir, format='zip', root_dir=data_dir, base_dir=land_name)
shutil.rmtree(land_dir)

# Create bathymetry